# MLflow Demo: A Comprehensive Guide

This notebook provides a **hands-on, beginner-friendly** demonstration of MLflow, an open-source platform for managing the end-to-end machine learning lifecycle. 

## 📚 What You'll Learn

By the end of this notebook, you will be able to:

1. ✅ **Track experiments** - Log parameters, metrics, and models systematically
2. ✅ **Use autologging** - Automatically capture ML metadata with minimal code
3. ✅ **Manage models** - Version control and lifecycle management via Model Registry
4. ✅ **Compare runs** - Evaluate different hyperparameter configurations side-by-side
5. ✅ **Log artifacts** - Save plots, data files, and other outputs
6. ✅ **Load models** - Retrieve and use previously trained models
7. ✅ **Query experiments** - Programmatically search and filter runs
8. ✅ **Serve models** - Deploy models for predictions (advanced)

## 🎯 Why MLflow?

MLflow solves common ML challenges:
- 📊 **Experiment tracking**: "Which hyperparameters gave the best results?"
- 🔄 **Reproducibility**: "How did I train this model 3 months ago?"
- 📦 **Model versioning**: "Which model is currently in production?"
- 🤝 **Team collaboration**: Share experiments and models with teammates
- 🚀 **Deployment**: Package and deploy models consistently

---

## 1. Setup

First, let's install the necessary libraries and import them. We'll use `scikit-learn` to train simple models and `mlflow` to manage the MLOps lifecycle.

In [24]:
# !uv pip install mlflow scikit-learn 

### ⚡ Getting Started Checklist

Before running this notebook:

- [ ] Install required packages (run the cell below)
- [ ] Open a terminal in this notebook's directory
- [ ] Run `mlflow ui` in the terminal to start the MLflow UI
- [ ] Open `http://127.0.0.1:5000` in your browser
- [ ] Keep the MLflow UI tab open while working through the notebook
- [ ] Refresh the UI after running cells to see new results

**Ready? Let's install the packages!**

In [25]:
# Core MLflow imports
import mlflow
import mlflow.sklearn

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Data handling and utilities
import pandas as pd
import numpy as np
import os

print("All libraries imported successfully!")
print(f"MLflow version: {mlflow.__version__}")

All libraries imported successfully!
MLflow version: 3.5.1


## Understanding MLflow Core Concepts

Before we dive into the code, let's understand the key concepts in MLflow:

### 🔬 **Experiment**
A logical grouping of related runs. Think of it as a project or research question (e.g., "Customer Churn Prediction" or "Image Classification v2"). All runs related to solving the same problem go into one experiment.

### 🏃 **Run**
A single execution of your model training code. Each time you train a model (even with different parameters), that's a new run. A run contains:
- **Parameters**: Input values that don't change during training (e.g., `learning_rate=0.01`, `n_estimators=100`)
- **Metrics**: Output values that measure performance (e.g., `accuracy=0.95`, `loss=0.23`)
- **Artifacts**: Files produced during the run (models, plots, data files, etc.)
- **Metadata**: Start time, end time, source code version, etc.

### 📊 **Parameters vs Metrics**
- **Parameters** are inputs you SET (hyperparameters, configurations)
- **Metrics** are outputs you MEASURE (accuracy, precision, loss)

### 📦 **Artifacts**
Any file you want to save: trained models, plots, preprocessors, feature importance charts, confusion matrices, etc.

### 🎯 **Model Registry**
A centralized repository for managing model versions and lifecycle stages (Staging → Production → Archived)

---

### 🎓 Quick Reference Card

Before you start, here's a cheat sheet of the most common MLflow commands:

```python
# Start tracking
mlflow.set_experiment("My Experiment")

# Log during training
with mlflow.start_run(run_name="my_run"):
    mlflow.log_param("learning_rate", 0.01)      # Log a parameter
    mlflow.log_metric("accuracy", 0.95)          # Log a metric
    mlflow.sklearn.log_model(model, "model")     # Log a model
    mlflow.log_artifact("plot.png")              # Log a file
    mlflow.set_tag("type", "baseline")           # Add metadata

# Autologging (easiest!)
mlflow.sklearn.autolog()  # Then just train normally

# Load models
model = mlflow.sklearn.load_model("runs:/RUN_ID/model")
model = mlflow.pyfunc.load_model("models:/ModelName/Production")

# Search runs
runs = mlflow.search_runs(experiment_names=["My Experiment"])
```

---

---

## 2. Experiment Tracking

MLflow Tracking allows you to log and query experiments. Let's break down the key components:

- **Experiment**: A logical grouping of related runs (e.g., all runs for a specific project)
- **Run**: A single execution of your model training code
- **Parameters**: Configuration values and hyperparameters
- **Metrics**: Performance measurements (accuracy, loss, etc.)
- **Artifacts**: Output files (models, plots, data, etc.)

### 2.1. Create an Experiment

First, we create (or connect to) an experiment. If an experiment with the same name already exists, MLflow will use that one.

In [26]:
# Set the experiment name - creates it if it doesn't exist, or uses existing one
experiment = mlflow.set_experiment("MLflow Demo")

print(f"Experiment Name: {experiment.name}")
print(f"Experiment ID: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")

Experiment Name: MLflow Demo
Experiment ID: 292277601767576213
Artifact Location: file:///Users/tarekatwan/Repos/MyWork/Teach/repos/adv_ml_ds/activities/mlflow/mlruns/292277601767576213


### 2.2. Start a Run and Log Parameters, Metrics, and Model

Within a `with mlflow.start_run():` block, you can:
- Log **parameters** (hyperparameters, configuration)
- Log **metrics** (performance measures)
- Log **models** (trained model objects)
- Add **tags** (metadata for organization)

The `with` statement automatically starts and ends the run, ensuring everything is properly recorded.

In [27]:
# Load Iris dataset
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Start an MLflow run with a descriptive name
with mlflow.start_run(run_name="Logistic Regression Baseline") as run:
    # Log parameters - these are inputs/hyperparameters
    params = {
        "solver": "lbfgs", 
        "max_iter": 1000, 
        "multi_class": "auto", 
        "random_state": 8888
    }
    mlflow.log_params(params)
    
    # You can also log individual parameters
    mlflow.log_param("model_type", "LogisticRegression")

    # Train the model
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)

    # Make predictions
    y_pred = lr.predict(X_test)

    # Log metrics - these are outputs/performance measures
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("test_samples", len(y_test))

    # Log the trained model
    mlflow.sklearn.log_model(lr, "logistic-regression-model")
    
    # Add tags to help organize and search runs later
    mlflow.set_tag("model_family", "linear")
    mlflow.set_tag("dataset", "iris")
    
    # Store the run_id for later use
    run_id = run.info.run_id

    print(f"✅ Run ID: {run_id}")
    print(f"📊 Accuracy: {accuracy:.4f}")
    print(f"🔗 Model URI: runs:/{run_id}/logistic-regression-model")

/Users/tarekatwan/Repos/MyWork/Teach/repos/adv_ml_ds/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/11/04 19:55:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 19:55:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:55:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run ID: 31ca7285e5504d86bfcaa62d8ece67a1
📊 Accuracy: 1.0000
🔗 Model URI: runs:/31ca7285e5504d86bfcaa62d8ece67a1/logistic-regression-model


### 🖥️ Viewing Your Results in the MLflow UI

Now let's see what we just logged! 

**Step 1: Start the MLflow UI**
Open a terminal in this notebook's directory and run:
```bash
mlflow ui
```

**Step 2: Open the UI in your browser**
Navigate to: **`http://127.0.0.1:5000`**

**What you'll see in the UI:**
1. **Experiments List** - All your experiments (you should see "MLflow Demo")
2. **Runs Table** - Click on your experiment to see all runs
3. **Run Details** - Click on a run to see:
   - **Parameters**: The hyperparameters you logged
   - **Metrics**: Performance metrics with values
   - **Artifacts**: Your saved model and any files
   - **Tags**: Metadata like model_family, dataset
   - **Source**: The code that created this run

**💡 Pro Tip:** Keep the MLflow UI open in a browser tab while working through this notebook. Refresh it after each section to see new runs appear!

---

---

### 2.3. Autologging - The Easy Way! 🚀

MLflow can automatically log parameters, metrics, and models for many popular libraries. This is the **easiest way** to get started with MLflow!

Supported libraries include: scikit-learn, TensorFlow, Keras, PyTorch, XGBoost, LightGBM, and more.

**Why learn manual tracking first?** Understanding what MLflow logs behind the scenes helps you debug issues and customize logging when needed. Now let's see how autologging makes your life easier!

In [28]:
# First, let's load the Iris dataset for this demo
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"📊 Dataset loaded: {len(X_train)} training samples, {len(X_test)} test samples")
print(f"🌸 Classes: {iris.target_names}")

# Enable autologging for scikit-learn
mlflow.sklearn.autolog()

# Now just train your model normally - MLflow handles the rest!
with mlflow.start_run(run_name="Decision Tree with Autolog"):
    dt = DecisionTreeClassifier(max_depth=3, random_state=42)
    dt.fit(X_train, y_train)
    
    # MLflow automatically logs:
    # - All model parameters (max_depth, criterion, etc.)
    # - Training score
    # - The model itself
    # - Feature importance (if available)
    
    # You can still manually log additional things
    mlflow.set_tag("notes", "Testing autologging feature")
    
    print("✅ Model trained! Check MLflow UI - everything was logged automatically!")

# Turn off autologging if you want manual control again
mlflow.sklearn.autolog(disable=True)

📊 Dataset loaded: 120 training samples, 30 test samples
🌸 Classes: ['setosa' 'versicolor' 'virginica']
✅ Model trained! Check MLflow UI - everything was logged automatically!
✅ Model trained! Check MLflow UI - everything was logged automatically!


---

## 3. Model Registry & Management 📦

The **Model Registry** is MLflow's solution for managing model versions and deployments. It provides:
- **Centralized model storage**: All models in one place
- **Version control**: Track different versions of the same model
- **Lifecycle management**: Move models through stages (Staging → Production)
- **Model lineage**: See which experiment/run created each model

### 3.1. Register a Model

To register a model, we need to reference it by its run ID and artifact path. Here's the **robust way** to do this using `mlflow.search_runs()`:

**Why this approach?** This cell can run independently even after a kernel restart, making your notebook more reliable!

In [29]:
# Find the run ID programmatically (robust approach)
# This works even if you restart your kernel and run this cell in isolation!
print("🔍 Searching for the 'Logistic Regression Baseline' run...")

# Use mlflow.search_runs() to find the run by name
# This is a real-world pattern - don't rely on in-memory variables!
baseline_run = mlflow.search_runs(
    experiment_names=["MLflow Demo"],
    filter_string="tags.`mlflow.runName` = 'Logistic Regression Baseline'",
    order_by=["start_time DESC"]  # Get most recent if multiple exist
).iloc[0]  # Get the first (most recent) match

run_id = baseline_run.run_id
print(f"✅ Found Run ID: {run_id}")

# Now construct the model URI and register it
model_uri = f"runs:/{run_id}/logistic-regression-model"

# Register the model with a meaningful name
model_name = "IrisClassifier"
registered_model = mlflow.register_model(model_uri, model_name)

print(f"\n✅ Model registered successfully!")
print(f"📦 Model Name: {model_name}")
print(f"🔢 Version: {registered_model.version}")
print(f"💡 This approach works even after kernel restart!")

Registered model 'IrisClassifier' already exists. Creating a new version of this model...
2025/11/04 19:55:22 WARNING mlflow.tracking._model_registry.fluent: Run with id 31ca7285e5504d86bfcaa62d8ece67a1 has no artifacts at artifact path 'logistic-regression-model', registering model based on models:/m-489d02a843af4e0fb5f743bb01797695 instead


🔍 Searching for the 'Logistic Regression Baseline' run...
✅ Found Run ID: 31ca7285e5504d86bfcaa62d8ece67a1

✅ Model registered successfully!
📦 Model Name: IrisClassifier
🔢 Version: 2
💡 This approach works even after kernel restart!


Created version '2' of model 'IrisClassifier'.


### 3.2. Manage Model Versions and Stages

The Model Registry provides lifecycle stages for your models:
- **None**: Newly registered models start here
- **Staging**: Models being tested before production
- **Production**: Models actively serving predictions
- **Archived**: Deprecated models kept for reference

Let's transition our model through these stages:

In [30]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Get the latest version of the model
latest_versions = client.get_latest_versions(name=model_name, stages=["None"])
if latest_versions:
    model_version = latest_versions[0].version
    
    # Transition to Staging first (best practice)
    client.transition_model_version_stage(
        name=model_name,
        version=model_version,
        stage="Staging"
    )
    print(f"✅ Model v{model_version} moved to Staging")
    
    # After testing, promote to Production
    client.transition_model_version_stage(
        name=model_name,
        version=model_version,
        stage="Production"
    )
    print(f"🚀 Model v{model_version} promoted to Production!")
else:
    print("⚠️ No model versions found in 'None' stage")

✅ Model v2 moved to Staging
🚀 Model v2 promoted to Production!


/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_6349/3323063058.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name, stages=["None"])
/var/folders/48/j6k669vx63qd_68k2_502cl40000gn/T/ipykernel_6349/3323063058.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
/var/folders/48/j6k669vx63qd_68k2_502cl4

In [31]:
# Load the production version of the registered model
# This ensures you always get the current production model, regardless of which run it came from
production_model_uri = "models:/IrisClassifier/Production"

production_model = mlflow.sklearn.load_model(production_model_uri)

# Make predictions
test_samples = X_test[:5]  # First 5 test samples
predictions = production_model.predict(test_samples)

print("✅ Production model loaded from Registry!")
print(f"\n📊 Sample Predictions:")
for i, (sample, pred) in enumerate(zip(test_samples, predictions)):
    print(f"  Sample {i+1}: {iris.target_names[pred]}")
    
print(f"\n💡 This model came from stage: Production")
print(f"🔗 Model URI: {production_model_uri}")

✅ Production model loaded from Registry!

📊 Sample Predictions:
  Sample 1: versicolor
  Sample 2: setosa
  Sample 3: virginica
  Sample 4: versicolor
  Sample 5: versicolor

💡 This model came from stage: Production
🔗 Model URI: models:/IrisClassifier/Production


### 3.3. Loading Models from the Registry 🔄

Now that we have a model registered and in Production, let's see how to load and use it. This is **critical** for production systems!

#### Method 1: Load by Stage (Recommended)

## 4. Hyperparameter Tuning Comparison

One of MLflow's most powerful features is comparing multiple runs side-by-side. This is incredibly useful for hyperparameter tuning!

**Nested Runs:** We'll use a parent run to group related experiments, with child runs for each hyperparameter configuration.

In [32]:
from sklearn.ensemble import RandomForestClassifier

# Parent run for the overall tuning experiment
with mlflow.start_run(run_name="Random Forest Hyperparameter Tuning") as parent_run:
    mlflow.set_tag("tuning_strategy", "grid_search")
    
    # Define hyperparameters to test
    n_estimators_list = [10, 50, 100]
    max_depth_list = [3, 5, None]
    
    best_accuracy = 0
    best_params = {}
    
    for n_estimators in n_estimators_list:
        for max_depth in max_depth_list:
            # Child run for each configuration (nested=True)
            with mlflow.start_run(
                run_name=f"RF_n{n_estimators}_d{max_depth}", 
                nested=True
            ) as child_run:
                # Log parameters
                mlflow.log_param("n_estimators", n_estimators)
                mlflow.log_param("max_depth", max_depth)
                mlflow.log_param("random_state", 42)
                
                # Train the model
                rf = RandomForestClassifier(
                    n_estimators=n_estimators, 
                    max_depth=max_depth,
                    random_state=42
                )
                rf.fit(X_train, y_train)
                
                # Make predictions
                y_pred = rf.predict(X_test)
                
                # Log metrics
                accuracy = accuracy_score(y_test, y_pred)
                mlflow.log_metric("accuracy", accuracy)
                
                # Track best model
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {"n_estimators": n_estimators, "max_depth": max_depth}
                
                # Log the model
                mlflow.sklearn.log_model(rf, "random-forest-model")
                
                print(f"n_estimators={n_estimators}, max_depth={max_depth} → Accuracy: {accuracy:.4f}")
    
    # Log best results to parent run
    mlflow.log_params(best_params)
    mlflow.log_metric("best_accuracy", best_accuracy)
    
    print(f"\n🏆 Best Configuration: {best_params}")
    print(f"🎯 Best Accuracy: {best_accuracy:.4f}")

2025/11/04 19:55:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 19:55:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:55:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:55:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 19:55:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


n_estimators=10, max_depth=3 → Accuracy: 1.0000


2025/11/04 19:55:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:55:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 19:55:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


n_estimators=10, max_depth=5 → Accuracy: 1.0000


2025/11/04 19:55:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:55:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 19:55:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

n_estimators=100, max_depth=3 → Accuracy: 1.0000


2025/11/04 19:55:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:55:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 19:55:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


n_estimators=100, max_depth=5 → Accuracy: 1.0000


2025/11/04 19:55:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


n_estimators=100, max_depth=None → Accuracy: 1.0000

🏆 Best Configuration: {'n_estimators': 10, 'max_depth': 3}
🎯 Best Accuracy: 1.0000


### 📊 Comparing Runs in the MLflow UI

Now go to the MLflow UI and let's compare these runs:

1. **Select Multiple Runs**: Check the boxes next to the runs you want to compare
2. **Click "Compare"**: You'll see a comparison view with:
   - Side-by-side parameter values
   - Metrics comparison (bar charts, scatter plots)
   - Ability to visualize parameter vs metric relationships
3. **Parallel Coordinates Plot**: Great for seeing which parameter combinations work best
4. **Scatter Plot**: Compare any two metrics or parameters

**💡 Try This:** In the UI, create a scatter plot with `n_estimators` on X-axis and `accuracy` on Y-axis to see the relationship!

---

## 5. Artifact Logging

MLflow allows you to log not just models, but also other artifacts like:
- 📊 **Plots**: Confusion matrices, ROC curves, feature importance charts
- 📁 **Data files**: Preprocessed datasets, train/test splits
- 📝 **Reports**: Model documentation, evaluation reports
- 🖼️ **Images**: Any visualizations you create

Let's create and log some useful artifacts.

In [33]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import json

with mlflow.start_run(run_name="Artifact Logging Demo"):
    # Train a quick model
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # 1. Log a confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names)
    disp.plot(cmap='Blues')
    plt.title("Confusion Matrix")
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()
    
    # 2. Log a JSON file with additional metadata
    metadata = {
        "dataset": "Iris",
        "n_samples_train": len(X_train),
        "n_samples_test": len(X_test),
        "features": iris.feature_names,
        "classes": iris.target_names.tolist()
    }
    with open("model_metadata.json", "w") as f:
        json.dump(metadata, f, indent=2)
    mlflow.log_artifact("model_metadata.json")
    
    # 3. Log a text report
    report = f"""
    Model Evaluation Report
    =======================
    Model Type: Logistic Regression
    Accuracy: {accuracy_score(y_test, y_pred):.4f}
    Dataset: Iris (multiclass classification)
    Training Samples: {len(X_train)}
    Test Samples: {len(X_test)}
    """
    with open("evaluation_report.txt", "w") as f:
        f.write(report)
    mlflow.log_artifact("evaluation_report.txt")
    
    # Clean up local files
    os.remove("confusion_matrix.png")
    os.remove("model_metadata.json")
    os.remove("evaluation_report.txt")
    
    print("✅ Logged confusion matrix, metadata JSON, and evaluation report!")
    print("📁 Check the 'Artifacts' tab in the MLflow UI to view them")

✅ Logged confusion matrix, metadata JSON, and evaluation report!
📁 Check the 'Artifacts' tab in the MLflow UI to view them


In the MLflow UI, click on any run and go to the **Artifacts** tab. You'll see your logged files and can download or preview them directly in the browser!

---

## 5.5. Loading and Using Logged Models 🔄

This is a **critical skill**: loading a previously trained model from MLflow and using it for predictions. This is how you'd use models in production!

### Method 1: Load by Run ID

In [34]:
# Get the run_id from the first run we created
# In practice, you'd get this from the MLflow UI or by querying
print(f"Original run ID: {run_id}")

# Load the model using the run ID
model_uri = f"runs:/{run_id}/logistic-regression-model"
loaded_model = mlflow.sklearn.load_model(model_uri)

# Make predictions with the loaded model
sample_data = [[5.1, 3.5, 1.4, 0.2]]  # A sample iris flower
prediction = loaded_model.predict(sample_data)
predicted_class = iris.target_names[prediction[0]]

print(f"✅ Model loaded successfully!")
print(f"🔮 Prediction for {sample_data}: {predicted_class}")

Original run ID: 31ca7285e5504d86bfcaa62d8ece67a1
✅ Model loaded successfully!
🔮 Prediction for [[5.1, 3.5, 1.4, 0.2]]: setosa


In [35]:
model_uri = f"models:/{model_name}/Production"
print(f"Model URI: {model_uri}")

Model URI: models:/IrisClassifier/Production


In [36]:
# Search for all runs in the current experiment
runs = mlflow.search_runs(
    experiment_names=["MLflow Demo"],
    order_by=["metrics.accuracy DESC"]  # Sort by accuracy, best first
)

print("📊 All Runs (sorted by accuracy):")
print(runs[["run_id", "tags.mlflow.runName", "params.model_type", "metrics.accuracy"]].head(10))

print("\n" + "="*60)

# Filter runs: Find all runs with accuracy > 0.95
high_accuracy_runs = mlflow.search_runs(
    experiment_names=["MLflow Demo"],
    filter_string="metrics.accuracy > 0.95"
)

print(f"\n🎯 High Accuracy Runs (>0.95): {len(high_accuracy_runs)} found")
if len(high_accuracy_runs) > 0:
    print(high_accuracy_runs[["tags.mlflow.runName", "metrics.accuracy"]].head())

print("\n" + "="*60)

# Get the best run
best_run = runs.iloc[0]  # First row after sorting by accuracy DESC
print(f"\n🏆 Best Run:")
print(f"   Run Name: {best_run['tags.mlflow.runName']}")
print(f"   Accuracy: {best_run['metrics.accuracy']:.4f}")
print(f"   Run ID: {best_run['run_id']}")

📊 All Runs (sorted by accuracy):
                             run_id           tags.mlflow.runName  \
0  dffe472959124fb087f05faeded263c3                 RF_n100_dNone   
1  a7dd5615610a454cb6a284f6fa192844                    RF_n100_d5   
2  be66fc6cd65047ebbfe63e2d8895bc59                    RF_n100_d3   
3  3801acd69db74b2fb2506f23f77c9e8f                  RF_n50_dNone   
4  df3a6acde846468e9a00133871da8d08                     RF_n50_d5   
5  7bceb0f38a6e4f6295d5de378df70f00                     RF_n50_d3   
6  3e2ef09a15e04d62b71f256155bcc647                  RF_n10_dNone   
7  bd6fda980d47435983726fc1d8a8f85f                     RF_n10_d5   
8  fd11a96f9e8a47f594462651028e1f94                     RF_n10_d3   
9  31ca7285e5504d86bfcaa62d8ece67a1  Logistic Regression Baseline   

    params.model_type  metrics.accuracy  
0                None               1.0  
1                None               1.0  
2                None               1.0  
3                None               1.0

In [37]:
import requests
import json

# Only run this if you have the MLflow server running on port 5001!
def call_model_api(sepal_length, sepal_width, petal_length, petal_width, port=5001):
    """
    Call the MLflow model serving API
    """
    url = f"http://127.0.0.1:{port}/invocations"
    
    # Format data according to MLflow's expected format
    data = {
        "dataframe_split": {
            "columns": ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"],
            "data": [[sepal_length, sepal_width, petal_length, petal_width]]
        }
    }
    
    headers = {"Content-Type": "application/json"}
    
    try:
        response = requests.post(url, json=data, headers=headers)
        if response.status_code == 200:
            prediction = response.json()
            return prediction
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Server not running or error: {e}"

# Example (will only work if server is running)
print("To test the REST API:")
print("1. Start server in terminal: mlflow models serve -m 'models:/IrisClassifier/Production' -p 5001 --env-manager=local")
print("2. Then run this cell to call the API")
print("\nExample API call:")
print(call_model_api(5.1, 3.5, 1.4, 0.2))

To test the REST API:
1. Start server in terminal: mlflow models serve -m 'models:/IrisClassifier/Production' -p 5001 --env-manager=local
2. Then run this cell to call the API

Example API call:
Server not running or error: HTTPConnectionPool(host='127.0.0.1', port=5001): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12feea300>: Failed to establish a new connection: [Errno 61] Connection refused'))


### Making REST API Calls (if you use the CLI server)

If you do start the MLflow server via terminal, here's how to call it:

## 6.5. Querying Experiments Programmatically 🔍

Instead of using the UI, you can search and filter runs programmatically. This is powerful for automation and analysis!

## 7. Model Serving (Optional Advanced Topic) 🚀

MLflow can serve models as REST APIs for production deployment. There are two main approaches:

### Option A: Command Line (for quick testing)

Open a **new terminal** and run:
```bash
cd /path/to/this/notebook/directory
mlflow models serve -m "models:/IrisClassifier/Production" -p 5001 --env-manager=local
```

**Note:** This is a blocking command - the terminal will be occupied while the server runs. Press CTRL+C to stop it.

### Option B: Python API (recommended for notebooks)

In [38]:
# Instead of serving via terminal, let's use the model directly in Python
# This is more practical for most use cases

# Load the production model
model = mlflow.pyfunc.load_model("models:/IrisClassifier/Production")

# Create a prediction function (simulates what a REST API would do)
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
    """
    Predict iris species given flower measurements
    """
    input_data = pd.DataFrame({
        'sepal length (cm)': [sepal_length],
        'sepal width (cm)': [sepal_width],
        'petal length (cm)': [petal_length],
        'petal width (cm)': [petal_width]
    })
    
    prediction = model.predict(input_data)
    species = iris.target_names[prediction[0]]
    
    return species

# Test the prediction function
result = predict_iris(5.1, 3.5, 1.4, 0.2)
print(f"🌸 Predicted species: {result}")

# Try a few more
print("\n📊 Batch Predictions:")
test_flowers = [
    (5.1, 3.5, 1.4, 0.2),
    (6.7, 3.1, 4.7, 1.5),
    (6.3, 2.9, 5.6, 1.8)
]

for i, (sl, sw, pl, pw) in enumerate(test_flowers, 1):
    species = predict_iris(sl, sw, pl, pw)
    print(f"  Flower {i}: {species}")

🌸 Predicted species: setosa

📊 Batch Predictions:
  Flower 1: setosa
  Flower 2: versicolor
  Flower 3: virginica


/Users/tarekatwan/Repos/MyWork/Teach/repos/adv_ml_ds/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/tarekatwan/Repos/MyWork/Teach/repos/adv_ml_ds/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/tarekatwan/Repos/MyWork/Teach/repos/adv_ml_ds/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/tarekatwan/Repos/MyWork/Teach/repos/adv_ml_ds/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


---

## 8. MLflow Best Practices & Tips 💡

### **For Students & Practitioners:**

1. **📝 Use Descriptive Run Names**
   - Bad: `run1`, `run2`, `test`
   - Good: `lr_baseline`, `rf_tuned_v2`, `final_production_model`

2. **🏷️ Tag Everything**
   - Use tags to categorize runs: `model_family`, `dataset_version`, `experiment_type`
   - Makes searching and filtering much easier later

3. **📊 Log What Matters**
   - Parameters: All hyperparameters that affect the model
   - Metrics: Not just accuracy - log precision, recall, F1, training time, etc.
   - Artifacts: Confusion matrices, feature importance plots, preprocessors

4. **🔄 Use Autologging When Possible**
   - Saves time and ensures consistency
   - Works great for scikit-learn, TensorFlow, PyTorch, XGBoost, etc.

5. **🎯 Model Registry Workflow**
   - Development → None stage
   - Testing complete → Staging stage
   - Production ready → Production stage
   - Deprecated → Archived stage

6. **🔍 Search & Compare Frequently**
   - Use the UI to compare runs visually
   - Use `mlflow.search_runs()` for programmatic analysis
   - Filter by metrics to find best models quickly

7. **📦 Log Complete Artifacts**
   - Don't just log the model - log the preprocessor, encoder, scaler, etc.
   - Future you will thank present you!

8. **🚫 Use a .gitignore File**
   - Your `mlruns/` folder can become very large with experiment data
   - **Don't commit it to Git!** Add `mlruns/` to your `.gitignore` file
   - This keeps your repository clean and avoids uploading large model files
   - Example `.gitignore` entry:
     ```
     mlruns/
     *.pyc
     __pycache__/
     ```

---

## 9. Common Issues & Troubleshooting 🔧

### Issue 1: "Cannot find MLflow runs"
**Solution:** Make sure you're running `mlflow ui` in the same directory where your notebook is, or where the `mlruns` folder exists.

### Issue 2: "Blank screen on MLflow UI"
**Solution:** Use `http://127.0.0.1:5000` instead of `http://localhost:5000`

### Issue 3: "Cannot load model - Run not found"
**Solution:** Check that the run_id is correct. Use `mlflow.search_runs()` to find valid run IDs.

### Issue 4: "Model Registry errors"
**Solution:** Make sure you've registered the model first with `mlflow.register_model()` before trying to transition stages.

### Issue 5: "Autologging not working"
**Solution:** 
- Call `mlflow.sklearn.autolog()` BEFORE training
- Make sure you're inside a `with mlflow.start_run():` block

---

## 10. Next Steps & Resources 🚀

### **What to explore next:**
1. **MLflow Projects**: Package your ML code for reproducibility
2. **MLflow with Deep Learning**: TensorFlow, PyTorch integration
3. **MLflow Deployment**: Deploy to cloud platforms (AWS SageMaker, Azure ML, etc.)
4. **Custom Models**: Create custom `pyfunc` models for complex pipelines
5. **MLflow Pipelines**: End-to-end ML pipelines with templates

### **Useful Resources:**
- 📖 [MLflow Documentation](https://mlflow.org/docs/latest/index.html)
- 🎓 [MLflow Tutorial](https://mlflow.org/docs/latest/tutorials-and-examples/index.html)
- 💬 [MLflow GitHub](https://github.com/mlflow/mlflow)
- 🎥 [MLflow Video Tutorials](https://www.youtube.com/results?search_query=mlflow+tutorial)

---

## Conclusion

Congratulations! 🎉 You've learned the essential features of MLflow:

✅ **Experiment Tracking** - Log parameters, metrics, and models  
✅ **Autologging** - Automatic logging for popular frameworks  
✅ **Model Registry** - Version control and lifecycle management  
✅ **Hyperparameter Tuning** - Compare multiple runs systematically  
✅ **Artifact Logging** - Save plots, data, and files  
✅ **Model Loading** - Load and use saved models  
✅ **Querying** - Search and filter runs programmatically  
✅ **Model Serving** - Deploy models as APIs  

**Remember:** MLflow is a tool to help you be more organized and productive in ML projects. Start simple, use autologging, and gradually incorporate more features as needed.

Happy modeling! 🚀